# Parsing List of Section 13F Securities
## Steps
 1. Parsing PDF URL from web page using BeautifulSoup
 2. Parsging PDF file to dataframe using tabula-py
 3. Processing dataframe using Pandas
 4. Saving dataframe to .csv
 5. Loading data to database using sqlalchemy
 6. Verification

In [101]:
import sys
import base64
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

## 1. Parsing PDF URL from web page

### 1.1 Retrieve page source

In [102]:
WEB_URL_BASE = 'https://www.sec.gov'
WEB_URL_13F = 'https://www.sec.gov/divisions/investment/13flists.htm'

In [103]:
def getPageSourceFromURL(weburl):
    req = urllib.request.Request(weburl)    
    response = urllib.request.urlopen(req)    
    the_page = response.read()
    type = sys.getfilesystemencoding()
    return the_page.decode(type)

pageSource = getPageSourceFromURL(WEB_URL_13F)
#print(pageSource)

### 1.2 Parse PDF url for this quarter

In [104]:
soup = BeautifulSoup(pageSource)
current_list_ul = soup.find("ul", class_="blue-chevron")
#print(current_list_ul)

In [105]:
current_list_a = current_list_ul.find("li").find("a")
#print(current_list_a)
pdf_url_relative = current_list_a['href']
#print(pdf_url_relative)

pdf_url = WEB_URL_BASE + pdf_url_relative
print(f'PDF URL: {pdf_url}')

PDF URL: https://www.sec.gov/divisions/investment/13f/13flist2019q3.pdf


## 2. Parsging PDF file to dataframe

In [106]:
import requests, PyPDF2, io

response = requests.get(pdf_url)

with io.BytesIO(response.content) as open_pdf_file:
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    num_pages = read_pdf.getNumPages()
print(num_pages)

527


In [107]:
from tabula import wrapper
df = wrapper.read_pdf(pdf_url,
                      pages=f'3-{num_pages}', 
                      guess=False, 
                      area=(132, 66, 720, 522), 
                      columns=(150, 162, 342, 468),
                      stream = True,
                      pandas_options={'header':None, 
                                      'names': ['CUSIP', 'OPTION_INDICATOR', 'ISSUER_NAME', 'ISSUER_DESCRIPTION', 'STATUS']}
                     )

In [108]:
df.shape

(17835, 5)

In [109]:
df.head()

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS
0,B38564 10 8,*,EURONAV NV ANTWERPEN,SHS,NaN
1,B38564 90 8,NaN,EURONAV NV ANTWERPEN,CALL,NaN
2,B38564 95 8,NaN,EURONAV NV ANTWERPEN,PUT,NaN
3,C0534L AA 3,NaN,AURORA CANNABIS INC,DEBT 5.500% 2/2,NaN
4,C10268 AC 1,NaN,BLACKBERRY LTD,DEBT 3.750%11/1,NaN


In [110]:
df.tail()

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS
17830,98986T 95 8,NaN,ZYNGA INC,PUT,NaN
17831,98986X 10 9,*,ZYNERBA PHARMACEUTICALS INC,COM,NaN
17832,98986X 90 9,NaN,ZYNERBA PHARMACEUTICALS INC,CALL,NaN
17833,98986X 95 9,NaN,ZYNERBA PHARMACEUTICALS INC,PUT,NaN
17834,NaN,NaN,NaN,Total Coun,"t: 17,834"


Check counters

In [111]:
total_count_text = df.tail(1)['STATUS'].to_string()
import re
total_count = int(re.findall("\d+", total_count_text)[0])
assert(total_count == df.shape[0]-1), f"Counters mismatch: {total_count} != {df.shape[0]-1}"

## 3. Processing dataframe

### 3.1 Reformat CUSIP NO by removing redundant sapces

In [112]:
df['CUSIP'] = df['CUSIP'].str.replace(' ', '')

### 3.2 Remove last row

In [113]:
df = df[:-1]

### 3.3 Check results

In [114]:
df.shape

(17834, 5)

In [115]:
df.tail()

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS
17829,98986T908,NaN,ZYNGA INC,CALL,NaN
17830,98986T958,NaN,ZYNGA INC,PUT,NaN
17831,98986X109,*,ZYNERBA PHARMACEUTICALS INC,COM,NaN
17832,98986X909,NaN,ZYNERBA PHARMACEUTICALS INC,CALL,NaN
17833,98986X959,NaN,ZYNERBA PHARMACEUTICALS INC,PUT,NaN


### 3.4 Raw data analysis

#### 3.4.1 Dump duplicated rows

In [116]:
df_duplicate_row = df[df.duplicated()]
df_duplicate_row

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS
1096,00162Q906,NaN,ALPS ETF TR,CALL,NaN
1097,00162Q956,NaN,ALPS ETF TR,PUT,NaN
2684,06746P908,NaN,BARCLAYS BK PLC,CALL,NaN
2685,06746P958,NaN,BARCLAYS BK PLC,PUT,NaN
2695,06746P903,NaN,BARCLAYS BK PLC,CALL,NaN
...,...,...,...,...,...
17529,97717W956,NaN,WISDOMTREE TR,PUT,NaN
17532,97717W901,NaN,WISDOMTREE TR,CALL,NaN
17533,97717W951,NaN,WISDOMTREE TR,PUT,NaN
17535,97717W909,NaN,WISDOMTREE TR,CALL,NaN


#### 3.4.2 Drop duplicates sand save it to df_unique

In [117]:
df_unique = df.drop_duplicates()
df_unique.shape

(17234, 5)

#### 3.4.3 Calculate number of duplicate rows

In [118]:
print(f'Number of duplicate records: {df.shape[0] - df_unique.shape[0]}, {df_duplicate_row.shape[0]}')

Number of duplicate records: 600, 600


#### 3.4.4 Dump duplicate CUSIP

In [119]:
df_duplicate_cusip = df_unique[df_unique.duplicated(subset='CUSIP')]
df_duplicate_cusip

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS
13188,74347B902,NaN,PROSHARES TR,CALL,DELETED
13189,74347B952,NaN,PROSHARES TR,PUT,DELETED
13421,74348A904,NaN,PROSHARES TR,CALL,NaN
13422,74348A954,NaN,PROSHARES TR,PUT,NaN


#### 3.4.5 Calculate number of duplicate CUSIPs

In [120]:
print(f'Number of duplicate CUSIP after dropping duplicate records: {df_duplicate_cusip.shape[0]}')

Number of duplicate CUSIP after dropping duplicate records: 4


## 4. Save to CSV file

In [121]:
pdf_file_name = pdf_url.split('/')[-1].replace('pdf', 'csv')
df.to_csv(index=False, path_or_buf=pdf_file_name)
print(f'Saved to file {pdf_file_name}')

Saved to file 13flist2019q3.csv


## 5. Load dataframe to HANA table

### 5.1 Add EFF_DATE column

In [122]:
# Get quater from pdf_file_name
QUARTER_END_DATES = ['03-31', '06-30', '09-30', '12-31']
year, quarter = pdf_file_name[-len('2019q3.csv') : -len('q3.csv')], pdf_file_name[-len('3.csv') : -len('.csv')]
quarter_end_date = QUARTER_END_DATES[int(quarter) - 1]

eff_date = f'{year}-{quarter_end_date}'

df['EFF_DATE'] = eff_date

In [123]:
df.head()

,CUSIP,OPTION_INDICATOR,ISSUER_NAME,ISSUER_DESCRIPTION,STATUS,EFF_DATE
0,B38564108,*,EURONAV NV ANTWERPEN,SHS,NaN,2019-09-30
1,B38564908,NaN,EURONAV NV ANTWERPEN,CALL,NaN,2019-09-30
2,B38564958,NaN,EURONAV NV ANTWERPEN,PUT,NaN,2019-09-30
3,C0534LAA3,NaN,AURORA CANNABIS INC,DEBT 5.500% 2/2,NaN,2019-09-30
4,C10268AC1,NaN,BLACKBERRY LTD,DEBT 3.750%11/1,NaN,2019-09-30


### 5.2 Create a connection to database

In [29]:
hana_host = 'my.sap.hana.host'
hana_port = 30215
hana_user = 'mydbuser'
hana_password = 'mypassword'
hana_tenant_db_name = 'MYBIP'

# use lower case for both schedma and name
hana_schema = 'vendor_data'
hana_table = 'sec_13f_securities'

In [30]:
from sqlalchemy import create_engine

connection_string = f'hana://{hana_user}:{hana_password}@{hana_host}:{hana_port}/{hana_tenant_db_name}'
engine = create_engine(connection_string)

### 5.3 Save data to database

In [31]:
df.to_sql(con=engine, schema=hana_schema, name=hana_table, if_exists='append', index=False, chunksize=10000)
print(f'{df.shape[0]} records have been inserted into HANA table {hana_schema.upper()}.{hana_table.upper()}')

17834 records have been inserted into HANA table VENDOR_DATA.SEC_13F_SECURITIES


## 6. Verification

### 6.1 Copy back data from HANA by querying against the EFF_DATE

In [32]:
query_statement = f"select * from {hana_schema}.{hana_table} where EFF_DATE='{eff_date}'"
df_hana = pd.read_sql_query(query_statement, engine)

### 6.2 Check number of records in database

In [33]:
df_hana.shape

(17834, 6)

### 6.3 Verifiy the data are correct

In [34]:
df_hana.head()

,cusip,option_indicator,issuer_name,issuer_description,status,eff_date
0,B38564108,*,EURONAV NV ANTWERPEN,SHS,None,2019-09-30
1,B38564908,None,EURONAV NV ANTWERPEN,CALL,None,2019-09-30
2,B38564958,None,EURONAV NV ANTWERPEN,PUT,None,2019-09-30
3,C0534LAA3,None,AURORA CANNABIS INC,DEBT 5.500% 2/2,None,2019-09-30
4,C10268AC1,None,BLACKBERRY LTD,DEBT 3.750%11/1,None,2019-09-30


In [35]:
df_hana.tail()

,cusip,option_indicator,issuer_name,issuer_description,status,eff_date
17829,98986T908,None,ZYNGA INC,CALL,None,2019-09-30
17830,98986T958,None,ZYNGA INC,PUT,None,2019-09-30
17831,98986X109,*,ZYNERBA PHARMACEUTICALS INC,COM,None,2019-09-30
17832,98986X909,None,ZYNERBA PHARMACEUTICALS INC,CALL,None,2019-09-30
17833,98986X959,None,ZYNERBA PHARMACEUTICALS INC,PUT,None,2019-09-30


In [36]:
# reset column names, it's all lower case names for df_hana.columns
df_hana.columns = df_hana.columns.str.upper()

# fill NA with 0 to use equals method
df = df.fillna(0)
df_hana = df_hana.fillna(0)

# change data type of Series to String to equals method
df['EFF_DATE'] = df['EFF_DATE'].apply(str)
df_hana['EFF_DATE'] = df_hana['EFF_DATE'].apply(str)

df_hana.equals(df)

True

# The End